In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

def poly_list_angles(poly):
    # Calculate the inside angles of polys
    poly_pad = np.pad(poly[:-1], [(1, 1), (0, 0)], 'wrap')
    edge = np.diff(poly_pad, axis=0)
    m1, m2 = edge[:-1], edge[1:]
    rad = math.pi + np.arctan2(np.cross(m1, m2), np.sum(m1 * m2, axis = 1))
    return rad

# def analyze_angles(gdf, tol):

#     collect_angles = []
#     count_angles = 0

#     for i in range(len(gdf)):

#       poly = gdf.iloc[i].geometry
#       poly = np.array(list(poly.exterior.coords))
#       angles = np.degrees(poly_list_angles(poly))

#       collect_angles.extend(list(angles))
#       count_angles += len(angles)

#     #plt.hist(collect_angles, bins=200)
#     #plt.show()

#     collect_angles = np.array(collect_angles)

#     mask_0 = (collect_angles > - tol) * (collect_angles < tol)
#     collect_angles = collect_angles[~ mask_0]
#     mask_180 = (collect_angles > 180 - tol) * (collect_angles < 180 + tol)
#     collect_angles = collect_angles[~ mask_180]

#     num_rect = np.sum((collect_angles > 90 - tol) * (collect_angles < 90 + tol))
#     num_rect_ = np.sum((collect_angles > 270 - tol) * (collect_angles < 270 + tol))

#     print(num_rect + num_rect_, count_angles, (num_rect+num_rect_) /len(collect_angles))

#     rang = (num_rect+num_rect_) /len(collect_angles)
#     return rang

def find_rect(poly, tol):
    rad = poly_list_angles(poly)
    angles = np.degrees(rad)
    angles[angles>180] = np.degrees(2*math.pi - rad)[angles>180]
    mask_rect_angles = (angles > 90 - tol) * (angles < 90 + tol)
    return mask_rect_angles

def plot_rect(poly, mask_rect_angles, ax = None):

    if ax != None:
        ax.plot(poly[:,0], poly[:,1], 'k')

        for vex, mask in zip(poly, mask_rect_angles):
          if mask:
              ax.plot(vex[0], vex[1], 'g.')
          else:
              ax.plot(vex[0], vex[1], 'r.')

    return ax

def rectangularity(gdf, tol, is_plot = True):

    if is_plot:
        plt.figure(figsize = (8,8)) # get axis handle
        ax = plt.axes()

    collect_all_rect = []
    for i in range(len(gdf)):

      poly = gdf.iloc[i].geometry
      poly = np.array(list(poly.exterior.coords))

      mask_rect_angles = find_rect(poly, tol)
      collect_all_rect.extend(mask_rect_angles)

      if is_plot:
          ax = plot_rect(poly, mask_rect_angles, ax)

    if is_plot:
        plt.axes(ax)
        plt.axis('equal')
        plt.show()

    rang = sum(collect_all_rect) / len(collect_all_rect)

    print("- Ratio of right:", sum(collect_all_rect) / len(collect_all_rect))
    print("Num. of edges that are not parallel:", sum(collect_all_rect), len(collect_all_rect))
    print("\n")

    return rang

In [ ]:
def find_parallel(rad, tol):

    # calc angles < 180
    angles = np.degrees(rad)
    angles[angles>180] = np.degrees(2*math.pi - rad)[angles>180]

    # Compare tolerance
    parallel_pair_mask = angles < tol

    return parallel_pair_mask, sum(parallel_pair_mask)>0

def plot_parallel(poly, is_found_parallel, ax = None):

    # Found pair, draw black
    if is_found_parallel:
        if ax != None:
            ax.plot(poly[:,0], poly[:,1], color='k')
        pass

    # Not found, draw red, ignoring very small segment
    else:
        if ax != None:
            ax.plot(poly[:,0], poly[:,1], color='r')
        pass
    return ax

def parallelism(gdf, tol, is_plot = True): # degree

    if is_plot:
      plt.figure(figsize = (8,8)) # get axis handle
      ax = plt.axes()

    count_polys = []
    collect_all_edges = []
    collect_all_pairs = []
    collect_all_len = []

    for id in range(len(gdf)): #

        poly =gdf.iloc[id].geometry
        poly = np.array(list(poly.exterior.coords))
        edges = np.diff(poly, axis=0)

        count_pairs = []
        count_edges = []

        for i in range(len(edges)):

            collect_angles = []
            m1 = edges[i]
            temp_edges = edges.copy()
            m2 = np.delete(temp_edges, i, axis=0)
            rad = math.pi + np.arctan2(np.cross(m1, m2), np.sum(m1 * m2, axis = 1))

            parallel_pair_mask, is_found_parallel = find_parallel(rad, tol)

            if is_plot:
                ax = plot_parallel(poly[i:i+2], is_found_parallel, ax = ax)

            count_edges.append(not is_found_parallel)
            count_pairs.extend(parallel_pair_mask)

            seq_len = np.sqrt(np.sum(np.square(m1)))
            collect_all_len.append(seq_len)

        collect_all_edges.extend(count_edges)
        collect_all_pairs.extend(count_pairs)
        count_polys.append(sum(count_edges) > 0)

    if is_plot:
        plt.axes(ax)
        plt.axis('equal')
        plt.show()

    redge = 1 - sum(collect_all_edges) / len(collect_all_edges)
    print("- Ratio of edges that are parallel:", 1 - sum(collect_all_edges) / len(collect_all_edges))
    print("Num. of edges that are not parallel:", sum(collect_all_edges), len(collect_all_edges))
    print("\n")

    rpair = 1 - sum(collect_all_pairs) / len(collect_all_pairs)
    print("- Ratio of pairs that are parallel:", 1 - sum(collect_all_pairs) / len(collect_all_pairs))
    print("Num. of pairs that are not parallel:", sum(collect_all_pairs), len(collect_all_pairs))
    print("\n")

    rpoly = 1 - sum(count_polys) / len(count_polys)
    print("- Ratio of polyons that are all parallel:", 1 - sum(count_polys) / len(count_polys))
    print("Num. of polyons that contains non-parallel:", sum(count_polys), len(count_polys))
    print("\n")

    acc_len = np.sum(np.array(collect_all_len)[collect_all_edges])
    rlen = 1 - acc_len / np.sum(collect_all_len)
    print("Ratio of edge length on non-parallel edges:", acc_len / np.sum(collect_all_len))
    print("Edge length on non-parallel edges:", acc_len)

    return redge, rpair, rpoly, rlen

In [ ]:
tol_thres = 4

In [ ]:
# Load the two shapefiles
shapefile1_path = "/content/drive/MyDrive/inferencee/shp/15k_target/15k_target_aligned.shp"

shapefile2_paths = ['/content/drive/MyDrive/inferencee/shp/15k_ResUnet/15k_target_aligned.shp',
                    '/content/drive/MyDrive/inferencee/shp/15k_GAN/15k_target_aligned.shp',
                    '/content/drive/MyDrive/inferencee/shp/15k_sd1.5/15k_target_aligned.shp',
                    '/content/drive/MyDrive/inferencee/shp/15k_sdxl/15k_target_aligned.shp',]

collect_measures = []

gdf1 = gpd.read_file(shapefile1_path).set_crs("epsg:25832", allow_override=True)

rang = rectangularity(gdf1, tol = tol_thres)

redge, _, rpoly, rlen = parallelism(gdf1, tol = tol_thres)
collect_measures.append([rang, redge, rlen, rpoly])


# Compare predictions from all models
for shapefile2_path in shapefile2_paths:

    gdf2 = gpd.read_file(shapefile2_path).set_crs("epsg:25832", allow_override=True)

    # Calculate difference
    symmetric_difference = gpd.overlay(gdf1, gdf2, how='symmetric_difference')
    #symmetric_difference.to_file(shapefile2_path + "symmetric_difference.shp")
    print(shapefile2_path, "Difference in m2:", symmetric_difference.area.sum())

    #symmetric_difference.plot()

    rang = rectangularity(gdf2, tol = tol_thres)

    redge, _, rpoly, rlen = parallelism(gdf2, tol = tol_thres)

    collect_measures.append([rang, redge, rlen, rpoly])

In [ ]:
for row in collect_measures:
  print(" & ".join(['{:.5f}'.format(elem) for elem in row]))